In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Load the data properly
df = pd.read_excel("/content/drive/MyDrive/real_estate.xlsx", skiprows=3)

# Rename columns
df.columns = [
    'Project', 'Stage', 'Category', 'Unit Type', 'Unit Code',
    'Grand Total', 'Total Finishing Price', 'Unit Total Price',
    'Planned Delivery Date', 'Actual Delivery Date', 'Completion Progress',
    'Land Area', 'Built Area', 'Basement Area', 'Uncovered Basement Area',
    'Penthouse Area', 'Semi Covered Roof', 'Roof Area', 'Garden Area',
    'Garage Area', 'Pergola Area', 'Storage Area', 'Extra BuiltUp Area',
    'Finishing Specs', 'Club Price'
]

# Clean data
df_clean = df.dropna(subset=['Unit Total Price', 'Category']).copy()

# Convert numeric columns
df_clean['Unit Total Price'] = pd.to_numeric(df_clean['Unit Total Price'], errors='coerce')
df_clean['Grand Total'] = pd.to_numeric(df_clean['Grand Total'], errors='coerce')

# Convert date column - using the more robust method
df_clean['Planned Delivery Date'] = pd.to_datetime(
    df_clean['Planned Delivery Date'].astype(str).str.split().str[0],
    errors='coerce'
)

# Drop rows where date conversion failed
df_clean = df_clean.dropna(subset=['Planned Delivery Date'])

# Verify cleaned data
print("Cleaned data shape:", df_clean.shape)
print("\nData types:")
print(df_clean.dtypes)
print("\nSample data:")
df_clean.head()

Mounted at /content/drive
Cleaned data shape: (1686, 25)

Data types:
Project                            object
Stage                              object
Category                           object
Unit Type                          object
Unit Code                          object
Grand Total                       float64
Total Finishing Price              object
Unit Total Price                  float64
Planned Delivery Date      datetime64[ns]
Actual Delivery Date               object
Completion Progress                object
Land Area                          object
Built Area                         object
Basement Area                      object
Uncovered Basement Area            object
Penthouse Area                     object
Semi Covered Roof                  object
Roof Area                          object
Garden Area                        object
Garage Area                        object
Pergola Area                       object
Storage Area                       object
Extra 

<ipython-input-3-36cddc66919a>:31: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_clean['Planned Delivery Date'] = pd.to_datetime(


,Project,Stage,Category,Unit Type,Unit Code,Grand Total,Total Finishing Price,Unit Total Price,Planned Delivery Date,Actual Delivery Date,...,Penthouse Area,Semi Covered Roof,Roof Area,Garden Area,Garage Area,Pergola Area,Storage Area,Extra BuiltUp Area,Finishing Specs,Club Price
1,Badya,Badya-District 2,Badya-Trio D-3 Bed-Garden,Apartment,D2A8-02-0101,12511000.0,3296412,15807412.0,2030-06-30,NaN,...,0,0,0,60.24,0,0,0,0,FFAI-BD-D2A8,320000
2,Badya,Badya-District 2,Badya-Trio D-3 Bed-Garden,Apartment,D2A8-02-0102,11954000.0,3030672,14984672.0,2030-06-30,NaN,...,0,0,0,70.61,0,0,0,0,FFAI-BD-D2A8,320000
3,Badya,Badya-District 2,Badya-Trio D-2 Bed-Typical,Apartment,D2A8-02-0112,9230000.0,2619084,11849084.0,2030-06-30,NaN,...,0,0,0,0,0,0,0,0,FFAI-BD-D2A8,320000
4,Badya,Badya-District 2,Badya-Trio D-2 Bed-Typical,Apartment,D2A8-02-0122,9230000.0,2619084,11849084.0,2030-06-30,NaN,...,0,0,0,0,0,0,0,0,FFAI-BD-D2A8,320000
5,Badya,Badya-District 2,Badya-Trio D-3 Bed-Typical,Apartment,D2A8-02-0131,10750000.0,3132230,13882230.0,2030-06-30,NaN,...,0,0,0,0,0,0,0,0,FFAI-BD-D2A8,320000


In [8]:
import pandas as pd
import numpy as np

# Load and prepare data
df = pd.read_excel('/content/drive/MyDrive/real_estate.xlsx', header=None)
header_row = df[df[0] == 'Project'].index[0]
df.columns = df.iloc[header_row]
df = df.iloc[header_row+1:].reset_index(drop=True)

# Clean and convert price data
price_col = 'Unit Total with Finishing Price'
df[price_col] = pd.to_numeric(df[price_col], errors='coerce')

# Convert from cents to dollars if needed
if df[price_col].median() > 1000000:
    df[price_col] = df[price_col] / 100

# Calculate variance - filtering out zeros and single-property categories
variance_results = (
    df.groupby('Category')[price_col]
    .agg(['count', 'var'])
    .reset_index()
    .rename(columns={'count': 'Property_Count', 'var': 'Variance'})
    .query('Variance > 0 and Property_Count > 1')  # This removes all zero-variance cases
    .sort_values('Variance', ascending=False)
)

# Format variance for readability
def format_variance(x):
    if x >= 1e12: return f"{x/1e12:.2f}T"
    if x >= 1e9: return f"{x/1e9:.2f}B"
    if x >= 1e6: return f"{x/1e6:.2f}M"
    return f"{x:,.0f}"

variance_results['Variance_Formatted'] = variance_results['Variance'].apply(format_variance)

# Display results
print("=== Properties With Price Variance ===")
print(variance_results[['Category', 'Property_Count', 'Variance_Formatted']].to_string(index=False))

# Export results
output_path = '/content/drive/MyDrive/property_variance_clean.xlsx'
variance_results.to_excel(output_path, index=False)
print(f"\nResults saved to: {output_path}")

=== Properties With Price Variance ===
                                    Category  Property_Count Variance_Formatted
                                 Garden View               2            106.17B
                Palmet NC-Third Floor-Office               9              1.92B
               Palmet NC-Fourth Floor-Office              10              1.85B
                    Alex-Ground Apartment-3B              12              1.12B
                    Alex-Ground Apartment-2B              28            828.61M
                Palmet NC-First Floor-Office               4            785.16M
                   Alex-Typical Apartment-2B              21            686.20M
                   Alex-Typical Apartment-3B              31            655.85M
                      PX-Town House X Middle              13            330.17M
                         Badya-Villa Type W2              11            297.04M
                       PX-Skye2-Penthouse-3B               4            286.29M
 